# Piping a RunnableParallel with Other Runnables

In [1]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [2]:
pip show langchain

Name: langchain
Version: 0.3.26
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: C:\Users\Marcus\anaconda3\envs\langchain_env_py312\Lib\site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext dotenv
%dotenv

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [5]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [6]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  seed = 365,
                  temperature = 0,
                  max_tokens = 500)

In [7]:
string_parser = StrOutputParser()

In [8]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [9]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [10]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup.\n2. Developing a simple Machine Learning model using Scikit-learn.\n3. Creating a basic web application using Django or Flask.'}

In [11]:
chain_time1 = (RunnableParallel({'books':chain_books, 
                                'projects':chain_projects}) 
              | chat_template_time 
              | chat 
              | string_parser
             )

In [12]:
chain_time2 = ({'books':chain_books, 
                'projects':chain_projects}
              | chat_template_time 
              | chat 
              | string_parser
             )

In [13]:
print(chain_time2.invoke({'programming language':'Python'}))

The time it would take to complete the literature and the projects can vary greatly depending on several factors such as your reading speed, comprehension level, the complexity of the projects, and the amount of time you can dedicate each day. 

However, here's a rough estimate:

1. "Fluent Python: Clear, Concise, and Effective Programming" - This book is around 800 pages. If you read and practice for about 2 hours a day, it might take you around 1-2 months to complete.

2. "Python Cookbook: Recipes for Mastering Python 3" - This book is around 700 pages. Again, assuming 2 hours of reading and practicing per day, it might take you around 1-2 months to complete.

3. "Effective Python: 90 Specific Ways to Write Better Python" - This book is around 230 pages. If you dedicate 2 hours a day, you could finish it in 2-3 weeks.

As for the projects:

1. Building a Web Scraper using BeautifulSoup - If you're familiar with the basics of web scraping, you could complete a simple project in a few 

In [14]:
chain_time2.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            